# Generate AMITT page metadata

Create a page for each of the AMITT objects, if it doesn't already exist.  If it does, update the metadata on it, but take care to preserve any human-generated notes. 

Todos:
* todo: add in the existing incident pages
* todo: add in the existing tactic notes from spreadsheets
* todo: add all framework comments to the repo issues list

In [1]:
# Libraries and functions
import pandas as pd
import numpy as np
import os

def create_incidentstring(techniqueid, it):

    incidentstr = '''
| Incident | Descriptions given for this incident |
| -------- | -------------------- |
'''
    incirow = '| [{0} {1}](../incidents/{0}.md) | {2} |\n'
    its = it[it['id_technique']==techniqueid]
    for index, row in its[['id_incident', 'name_incident']].drop_duplicates().sort_values('id_incident').iterrows():
        techstring = ', '.join(its[its['id_incident']==row['id_incident']]['name'].to_list())
        incidentstr += incirow.format(row['id_incident'], row['name_incident'], techstring)
    return incidentstr


def create_techstring(techlist):

    techstr = '''
| Technique | Description given for this incident |
| --------- | ------------------------- |
'''

    techrow = '| [{0} {1}](../techniques/{0}.md) | {2} {3} |\n'
    for index, row in techlist.sort_values('id_technique').iterrows():
        techstr += techrow.format(row['id_technique'], row['name_technique'], 
                                  row['id'], row['name'])
    return techstr


def create_taskstring(tasklist):

    taskstr = '''
| Task |
| ---- |
'''
    taskrow = '| [{0} {1}](../tasks/{0}.md) |\n'
    for index, row in tasklist.sort_values('id').iterrows():
        taskstr += taskrow.format(row['id'], row['name'])
    return taskstr


def create_techtacstring(techlist):

    techstr = '''
| Technique |
| --------- |
'''
    techrow = '| [{0} {1}](../techniques/{0}.md) |\n'
    for index, row in techlist.sort_values('id').iterrows():
        techstr += techrow.format(row['id'], row['name'])
    return techstr

In [2]:
# Load metadata from file
xlsx = pd.ExcelFile('amitt_metadata_v3.xlsx')
metadata = {}
for sheetname in xlsx.sheet_names:
    metadata[sheetname] = xlsx.parse(sheetname)
print('loaded {}'.format(list(metadata.keys())))


# Generate full cross-table between incidents and techniques
it = metadata['incidenttechniques']
it.index=it['id']
it = it['techniques'].str.split(',').apply(lambda x: pd.Series(x)).stack().reset_index(level=1, drop=True).to_frame('technique').reset_index().merge(it.drop('id', axis=1).reset_index()).drop('techniques', axis=1)
it = it.merge(metadata['incidents'][['id','name']], 
              left_on='incident', right_on='id',
              suffixes=['','_incident']).drop('incident', axis=1)
it = it.merge(metadata['techniques'][['id','name']], 
              left_on='technique', right_on='id',
              suffixes=['','_technique']).drop('technique', axis=1)

# Create individual tables and dictionaries
phases = metadata['phases']
techniques = metadata['techniques']
tasks = metadata['tasks']
incidents = metadata['incidents']

loaded ['phases', 'tactics', 'techniques', 'tasks', 'incidents', 'incidenttechniques']


In [3]:
# Generate datafiles
warntext = 'DO NOT EDIT ABOVE THIS LINE - PLEASE ADD NOTES BELOW'
warnlen = len(warntext)

for entity in ['phase', 'tactic', 'technique', 'task', 'incident']:
    entities = entity + 's'
    entitydir = '../{}'.format(entities)
    if not os.path.exists(entitydir):
        os.makedirs(entitydir)
    
    template = open('template_{}.md'.format(entity)).read()
    dfentity = metadata[entities]
    for index, row in dfentity[dfentity['name'].notnull()].iterrows():
        
        # First read in the file - if it exists - and grab everything 
        # below the "do not write about this line". Will write this 
        # out below new metadata. 
        datafile = '../{}/{}.md'.format(entities, row['id'])
        oldmetatext = ''
        if os.path.exists(datafile):
            with open(datafile) as f:
                filetext = f.read()
            warnpos = filetext.find(warntext)
            if warnpos == -1:
                print('no warning text found in {}: adding to file'.format(datafile))
                usertext = filetext
            else:
                oldmetatext = filetext[:warnpos+warnlen]
                usertext = filetext[warnpos+warnlen:]
        else:
            usertext = ''
        
        # Now populate datafiles with new metadata plus old userdata
        if entity == 'phase':
            metatext = template.format(id=row['id'], name=row['name'], summary=row['summary'])
        if entity == 'tactic':
            metatext = template.format(id=row['id'], name=row['name'],
                                       phase=row['phase'], summary=row['summary'],
                                       tasks=create_taskstring(tasks[tasks['tactic']==row['id']]),
                                       techniques=create_techtacstring(techniques[techniques['tactic']==row['id']]))
        if entity == 'task':
            metatext = template.format(id=row['id'], name=row['name'],
                                       tactic=row['tactic'], summary=row['summary'])
        if entity == 'technique':
            metatext = template.format(id=row['id'], name=row['name'],
                                       tactic=row['tactic'], summary=row['summary'],
                                       incidents=create_incidentstring(row['id'], it))
        if entity == 'incident':
            metatext = template.format(id=row['id'], name=row['name'],
                                       type=row['type'], summary=row['summary'],
                                       yearstarted=row['Year Started'], 
                                       fromcountry=row['From country'],
                                       tocountry=row['To country'],
                                       foundvia=row['Found via'],
                                       dateadded=row['When added'],
                                       techniques=create_techstring(it[it['id_incident'] == row['id']]))

        # Make sure the user data goes in
        if (metatext + warntext) != oldmetatext:
            print('Updating {}'.format(datafile))
            with open(datafile, 'w') as f:
                f.write(metatext)
                f.write(warntext)
                f.write(usertext)
                f.close()

Updating ../tactics/TA07.md
Updating ../techniques/T0001.md
Updating ../techniques/T0002.md
Updating ../techniques/T0003.md
Updating ../techniques/T0004.md
Updating ../techniques/T0005.md
Updating ../techniques/T0006.md
Updating ../techniques/T0007.md
Updating ../techniques/T0008.md
Updating ../techniques/T0009.md
Updating ../techniques/T0010.md
Updating ../techniques/T0011.md
Updating ../techniques/T0012.md
Updating ../techniques/T0014.md
Updating ../techniques/T0015.md
Updating ../techniques/T0016.md
Updating ../techniques/T0017.md
Updating ../techniques/T0018.md
Updating ../techniques/T0019.md
Updating ../techniques/T0020.md
Updating ../techniques/T0021.md
Updating ../techniques/T0022.md
Updating ../techniques/T0023.md
Updating ../techniques/T0024.md
Updating ../techniques/T0025.md
Updating ../techniques/T0026.md
Updating ../techniques/T0027.md
Updating ../techniques/T0028.md
Updating ../techniques/T0029.md
Updating ../techniques/T0030.md
Updating ../techniques/T0031.md
Updating ../

In [4]:
for technique in metadata['techniques']['id'].to_list():
    print('{}'.format(create_incidentstring(technique, it)))


| Incident | Descriptions given for this incident |
| -------- | -------------------- |
| [I00029 MH17 investigation](../incidents/I00029.md) | 4 D's (dismiss, distort, distract, dismay...Official channels ie: Embassies & Defence Ministry; TASS |
| [I00047 Sea of Azov](../incidents/I00047.md) | 4 D's (dismiss, distort, distract, dismay...Official channels ie: Embassies & Defence Ministry; TASS, Russian FSB security service blamed Ukraine for sparking the clashes, saying their "irrefutable" evidence would "soon be made public |
| [I00049 White Helmets: Chemical Weapons](../incidents/I00049.md) | 4 D's (dismiss, distort, distract, dismay...Official channels ie: Embassies & Defence Ministry; TASS |
| [I00053 China Huawei CFO Arrest](../incidents/I00053.md) | China also plays victim, innocence, plays by rules, misunderstood narrative, Chinese ambassador Lu Shaye accused Canada of applying a double standard, and has decried what he sees as “Western egotism and white supremacy” in the treat


| Incident | Descriptions given for this incident |
| -------- | -------------------- |
| [I00006 Columbian Chemicals](../incidents/I00006.md) | bait journalists/media/politicians |
| [I00010 ParklandTeens](../incidents/I00010.md) | journalist/media baiting |
| [I00015 ConcordDiscovery](../incidents/I00015.md) | journalist/media baiting |


| Incident | Descriptions given for this incident |
| -------- | -------------------- |
| [I00029 MH17 investigation](../incidents/I00029.md) | Demand insurmountable proof |
| [I00047 Sea of Azov](../incidents/I00047.md) | Demand insurmountable proof |


| Incident | Descriptions given for this incident |
| -------- | -------------------- |


| Incident | Descriptions given for this incident |
| -------- | -------------------- |


| Incident | Descriptions given for this incident |
| -------- | -------------------- |
| [I00006 Columbian Chemicals](../incidents/I00006.md) | Use SMS/text messages |


| Incident | Descriptions given for this incident 


| Incident | Descriptions given for this incident |
| -------- | -------------------- |


| Incident | Descriptions given for this incident |
| -------- | -------------------- |


| Incident | Descriptions given for this incident |
| -------- | -------------------- |


| Incident | Descriptions given for this incident |
| -------- | -------------------- |


| Incident | Descriptions given for this incident |
| -------- | -------------------- |


| Incident | Descriptions given for this incident |
| -------- | -------------------- |


| Incident | Descriptions given for this incident |
| -------- | -------------------- |


| Incident | Descriptions given for this incident |
| -------- | -------------------- |


| Incident | Descriptions given for this incident |
| -------- | -------------------- |


| Incident | Descriptions given for this incident |
| -------- | -------------------- |


| Incident | Descriptions given for this incident |
| -------- | -------------------- |


| Inciden

In [5]:
for incident in metadata['incidents']['id'].to_list():
    print('{}'.format(create_techstring(it[it['id_incident'] == incident])))


| Technique | Description given for this incident |
| --------- | ------------------------- |


| Technique | Description given for this incident |
| --------- | ------------------------- |
| [T0017 Promote online funding](../techniques/T0017.md) | I00002T002 Promote "funding" campaign |
| [T0018 Paid targeted ads](../techniques/T0018.md) | I00002T001 buy FB targeted ads |
| [T0019 Generate information pollution](../techniques/T0019.md) | I00002T003 create web-site - information pollution |
| [T0046 Search Engine Optimization](../techniques/T0046.md) | I00002T004 SEO optimisation/manipulation ("key words") |
| [T0056 Dedicated channels disseminate information pollution](../techniques/T0056.md) | I00002T003 create web-site - information pollution |
| [T0058 Legacy web content](../techniques/T0058.md) | I00002T005 legacy web content |
| [T0058 Legacy web content](../techniques/T0058.md) | I00002T006 hard to remove content and/or campaign/exploit TOS |


| Technique | Description given f


| Technique | Description given for this incident |
| --------- | ------------------------- |


| Technique | Description given for this incident |
| --------- | ------------------------- |


| Technique | Description given for this incident |
| --------- | ------------------------- |
| [T0007 Create fake Social Media Profiles / Pages / Groups](../techniques/T0007.md) | I00032T003 Fake FB groups/pages/profiles  |
| [T0010 Cultivate useful idiots](../techniques/T0010.md) | I00032T009 cultivate, manipulate, exploit useful idiots (Alex Jones... drives conspiracy theories) |
| [T0019 Generate information pollution](../techniques/T0019.md) | I00032T005 RT & Sputnik generate information pollution |
| [T0020 Trial content](../techniques/T0020.md) | I00032T007 4Chan/8Chan - trial content |
| [T0031 YouTube](../techniques/T0031.md) | I00032T006 YouTube; Reddit; (Instagram, LinkedIn; Pinterest; WhatsApp?) |
| [T0032 Reddit](../techniques/T0032.md) | I00032T006 YouTube; Reddit; (Instagram, Linke



| Technique | Description given for this incident |
| --------- | ------------------------- |


| Technique | Description given for this incident |
| --------- | ------------------------- |


| Technique | Description given for this incident |
| --------- | ------------------------- |
| [T0007 Create fake Social Media Profiles / Pages / Groups](../techniques/T0007.md) | I00056T003 Fake FB groups/pages/profiles + dark content (non-paid advertising) |
| [T0008 Create fake or imposter news sites](../techniques/T0008.md) | I00056T006 Fake news/synthetic web-sites |
| [T0021 Memes](../techniques/T0021.md) | I00056T005 Memes... anti-Isreal/USA/West, conspiracy narratives |
| [T0022 Conspiracy narratives](../techniques/T0022.md) | I00056T005 Memes... anti-Isreal/USA/West, conspiracy narratives |
| [T0046 Search Engine Optimization](../techniques/T0046.md) | I00056T004 SEO optimisation/manipulation ("key words") |
| [T0053 Twitter trolls amplify and manipulate](../techniques/T0053.md) | I000

## Make the grid

In [6]:
# Create the master grid that we make all the framework visuals from
# cols = number of tactics
# rows = max number of techniques per tactic + 2
phasedict = pd.Series(phases.name.values,index=phases.id).to_dict()
techdict = pd.Series(techniques.name.values,index=techniques.id).to_dict()

tactechs = techniques.groupby('tactic')['id'].apply(list).reset_index().rename({'id':'techniques'}, axis=1)
maxtechs = max(tactechs['techniques'].apply(len))
tactics = metadata['tactics'].merge(tactechs, left_on='id', right_on='tactic', how='left').fillna('').drop('tactic', axis=1)
tacdict = pd.Series(tactics.name.values,index=tactics.id).to_dict()

nrows, ncols = (maxtechs+2, len(tactics))
arr = [['' for i in range(ncols)] for j in range(nrows)] 
for index, tactic in tactics.iterrows():
    arr[0][index] = tactic['phase']
    arr[1][index] = tactic['id']
    if tactic['techniques'] == '':
        continue
    for index2, technique in enumerate(tactic['techniques']):
        arr[index2+2][index] = technique

#Save grid to file
matrixdir = '../matrices'
if not os.path.exists(matrixdir):
    os.makedirs(matrixdir)
pd.DataFrame(arr).to_csv(matrixdir + '/matrix_arr.csv', index=False, header=False)

arr

[['P01',
  'P01',
  'P02',
  'P02',
  'P02',
  'P02',
  'P02',
  'P03',
  'P03',
  'P03',
  'P03',
  'P04'],
 ['TA01',
  'TA02',
  'TA03',
  'TA04',
  'TA05',
  'TA06',
  'TA07',
  'TA08',
  'TA09',
  'TA10',
  'TA11',
  'TA12'],
 ['T0001',
  'T0005',
  'T0007',
  'T0010',
  'T0016',
  'T0019',
  'T0029',
  'T0039',
  'T0047',
  'T0057',
  'T0058',
  ''],
 ['T0002',
  'T0006',
  'T0008',
  'T0011',
  'T0017',
  'T0020',
  'T0030',
  'T0040',
  'T0048',
  'T0061',
  'T0059',
  ''],
 ['T0003',
  '',
  'T0009',
  'T0012',
  'T0018',
  'T0021',
  'T0031',
  'T0041',
  'T0049',
  '',
  'T0060',
  ''],
 ['T0004',
  '',
  '',
  'T0013',
  '',
  'T0022',
  'T0032',
  'T0042',
  'T0050',
  '',
  '',
  ''],
 ['', '', '', 'T0014', '', 'T0023', 'T0033', 'T0043', 'T0051', '', '', ''],
 ['', '', '', 'T0015', '', 'T0024', 'T0034', 'T0044', 'T0052', '', '', ''],
 ['', '', '', '', '', 'T0025', 'T0035', 'T0045', 'T0053', '', '', ''],
 ['', '', '', '', '', 'T0026', 'T0036', 'T0046', 'T0054', '', '', ''],

In [7]:
# Write HTML version of framework diagram to markdown file

html = '''# AMITT Latest Framework:

<table border="1">
<tr>
'''

for col in range(ncols):
    html += '<td><a href="phases/{0}.md">{0} {1}</a></td>'.format(
        arr[0][col], phasedict[arr[0][col]])
html += '</tr>\n'

html += '<tr style="background-color:blue;color:white;">'
for col in range(ncols):
    html += '<td><a href="tactics/{0}.md">{0} {1}</a></td>'.format(
        arr[1][col], tacdict[arr[1][col]])
html += '</tr>\n<tr>'

for row in range(2,nrows):
    for col in range(ncols):
        if arr[row][col] == '':
            html += '<td> </td>'
        else:
            html += '<td><a href="techniques/{0}.md">{0} {1}</a></td>'.format(
                arr[row][col], techdict[arr[row][col]])
    html += '</tr>\n<tr>'
html += '</tr></table>'
with open('../matrix.md', 'w') as f:
    f.write(html)

In [8]:
# Write HTML version of incident list to markdown file

html = '''# AMITT Incidents:

<table border="1">
<tr>
'''

cols = ['name', 'type', 'Year Started', 'From country', 'To country',
        'Found via']

html += '<th>{}</th>'.format('id')
for col in cols:
    html += '<th>{}</th>'.format(col)
html += '</tr>\n'

for index, row in incidents[incidents['name'].notnull()].iterrows():
    html += '\n<tr>'
    html += '<td><a href="incidents/{0}.md">{0}</a></td>'.format(row['id'])
    for col in cols:
            html += '<td>{}</td>'.format(row[col])
    html += '</tr>\n'
html += '</table>'
with open('../incidents.md', 'w') as f:
    f.write(html)

In [9]:
# Write clickable html version of the matrix grid to html file

html = '''<!DOCTYPE html>
<html>
<head>
	<title>AMITT</title>
</head>
<body>

<script>
function handleTechniqueClick(box) {
  var technique = document.getElementById(box);
  var checkBox = document.getElementById(box+"check");
  var text = document.getElementById(box+"text");
  if (checkBox.checked == true){
    text.style.display = "block";
    technique.bgColor = "Lime"
  } else {
     text.style.display = "none";
     technique.bgColor = "Silver"
  }
}
</script>


<h1>AMITT</h1>

<table border=1 bgcolor=silver>
'''

html += '<tr bgcolor=fuchsia>'
for col in range(ncols):
    html += '<td>{0} {1}</td>'.format(arr[0][col], phasedict[arr[0][col]])
html += '</tr>\n'

html += '<tr bgcolor=aqua>'
for col in range(ncols):
    html += '<td>{0} {1}</td>'.format(arr[1][col], tacdict[arr[1][col]])
html += '</tr>\n'

liststr = ''
html += '<tr>'
for row in range(2,nrows):
    for col in range(ncols):
        techid = arr[row][col]
        if techid == '':
            html += '<td bgcolor=white> </td>'
        else:
            html += '<td id="{0}">{0} {1}<input type="checkbox" id="{0}check"  onclick="handleTechniqueClick(\'{0}\')"></td>'.format(
                techid, techdict[techid])
            liststr += '<li id="{0}text" style="display:none">{0}: {1}</li>\n'.format(
                techid, techdict[techid])

    html += '</tr>\n<tr>'
html += '</tr></table><hr>'

html += '<ul>\n{}</ul>\n'.format(liststr)
html += '''
</body>
</html>
'''

with open('../matrix_to_message.html', 'w') as f:
    f.write(html)

In [11]:
tactics

,id,name,phase,rank,summary,techniques
0,TA01,Strategic Planning,P01,1,Defining the desired end state that is the set...,"[T0001, T0002, T0003, T0004]"
1,TA02,Objective Planning,P01,2,"Objectives should be clearly defined, measurab...","[T0005, T0006]"
2,TA03,Develop People,P02,3,"Develop online and offline users and agents, i...","[T0007, T0008, T0009]"
3,TA04,Develop Networks,P02,4,Develop online and offline communities and tra...,"[T0010, T0011, T0012, T0013, T0014, T0015]"
4,TA05,Microtargeting,P02,5,Target very specific populations of people,"[T0016, T0017, T0018]"
5,TA06,Develop Content,P02,6,Create and acquire content used in incident,"[T0019, T0020, T0021, T0022, T0023, T0024, T00..."
6,TA07,Channel Selection,P02,7,"Set up specific delivery, amplification and ma...","[T0029, T0030, T0031, T0032, T0033, T0034, T00..."
7,TA08,Pump Priming,P03,8,"Release content on a targetted small scale, pr...","[T0039, T0040, T0041, T0042, T0043, T0044, T00..."
8,TA09,Exposure,P03,9,Release content to general public or push to l...,"[T0047, T0048, T0049, T0050, T0051, T0052, T00..."
9,TA10,Go Physical,P03,10,Move incident into offline world,"[T0057, T0061]"
